In [1]:
import sys, time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import kornia
import math

In [6]:
K = 5
J = 4
N = 3
B = 1
magn = 5
grad = True
#預設機率與強度超參數的初始值
magnNorm = torch.ones(1)*magn/10.0 # normalize to 10 like in RandAugment
probNorm = torch.ones(1)*1/(K-2) # 1/(K-2) probability
magnLogit = torch.log(magnNorm/(1-magnNorm)) # convert to logit
probLogit = torch.log(probNorm/(1-probNorm)) # convert to logit
# affine transforms (mid, range)
angle = [0.0, 30.0] # [-30.0:30.0] rotation angle
trans = [0.0, 0.45] # [-0.45:0.45] X/Y translate
shear = [0.0, 0.30] # [-0.30:0.30] X/Y shear
scale = [1.0, 0.50] # [ 0.50:1.50] X/Y scale
# color transforms (mid, range)
bri = [0.0, 0.9] # [-0.9:0.9] brightness
con = [1.0, 0.9] # [0.1:1.9] contrast
sat = [0.1, 1.9] # [-0.30:0.30] saturation
#hue = [1.0, 0.50] # [ 0.70:1.30] hue
#gam = [1.0, 0.50] # [ 0.70:1.30] gamma
# actP: 機率的激活函數，actM: 強度的激活函數
actP = nn.Sigmoid()
actM = nn.Sigmoid()
# 建立機率與強度的可學習超參數矩陣 shape = (擴增方法數量*資料數量)
paramP = nn.Parameter(probLogit*torch.ones(K+J,N), requires_grad=grad)
paramM = nn.Parameter(magnLogit*torch.ones(K+J,N), requires_grad=grad)

In [7]:
idx = [0]
paramPos = torch.log(    actP(paramP[:,idx])) # [-Inf:0] [擴增方法數, data point number] 取針對該資料的使用機率之參數
paramNeg = torch.log(1.0-actP(paramP[:,idx])) # [-Inf:0] [擴增方法數, data point number] 取針對該資料的不使用機率之參數
paramM = actM(paramM[:,idx]) # (K+J)xB [0:1], default=magn

In [8]:
paramP = torch.cat([paramPos.view(-1,1), paramNeg.view(-1,1)], dim=1) # B*(K+J)x2
# reparametrize probabilities and magnitudes
sampleP = F.gumbel_softmax(paramP, tau=1.0, hard=True) # B*(K+J)x2
sampleP = sampleP[:,0]
sampleP = sampleP.reshape(K+J,B)
# # reparametrize magnitudes
# #sampleM = paramM[:K] * torch.rand(K,B).to(device) # KxB, prior: U[0,1]
sampleM = paramM[:K] * torch.randn(K,B) # KxB, prior: N(0,1)

In [9]:
R: torch.tensor = torch.zeros(B,3,3) + torch.eye(3)

In [10]:
ANG: torch.tensor = sampleP[0] * sampleM[0] * angle[1]

In [12]:
W=10
H=10

In [13]:
CTR: torch.tensor = torch.cat((W*torch.ones(B)//2, H*torch.ones(B)//2)).view(-1,2)

/home/rayeh/miniconda3/envs/autodo/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  """Entry point for launching an IPython kernel.


In [14]:
# define the scale factor
SCL: torch.tensor = torch.zeros_like(CTR)

In [19]:
SCL[:,0] = scale[0] + sampleP[3] * sampleM[3] * scale[1] # mid + B(0/1)*U[0,1]*M/10
SCL[:,1] = scale[0] + sampleP[4] * sampleM[4] * scale[1] # mid + B(0/1)*U[0,1]*M/10
R[:,0:2] = kornia.get_rotation_matrix2d(CTR, ANG, SCL)

In [20]:
T: torch.tensor = torch.zeros_like(R) + torch.eye(3)
T[:,0,2] = W * sampleP[1] * sampleM[1] * trans[1]
T[:,1,2] = H * sampleP[2] * sampleM[2] * trans[1]